In [1]:
import pandas as pd
from pathlib import Path

BASE = Path("../data/raw")
OUT  = Path("../data/intermediate")
OUT.mkdir(parents=True, exist_ok=True)

YEARS = [2019, 2020, 2021, 2022, 2023]

def detect_col(df, candidates):
    """Return the first column name that exists in df from candidates."""
    for c in candidates:
        if c in df.columns:
            return c
    return None

In [2]:
def filter_drivers(person_df):
    """
    Try multiple ways to restrict to drivers only.
    If we can't confidently detect the driver indicator, return unchanged and warn.
    """
    p = person_df.copy()

    # Common in NHTSA PERSON files
    per_typ = detect_col(p, ["PER_TYP", "PERTYP"])
    if per_typ:
        # Typically 1=Driver in NHTSA coding; verify by looking at PER_TYPNAME if present
        return p[p[per_typ] == 1]

    seat_pos = detect_col(p, ["SEAT_POS", "SEATPOS"])
    if seat_pos:
        # Driver seat often coded as 11 in many NHTSA datasets (front-left).
        # If this yields too few rows, we'll adjust after quick checks.
        return p[p[seat_pos] == 11]

    # If there is a name/label column we can use:
    per_typ_name = detect_col(p, ["PER_TYPNAME", "PERTYPNAME"])
    if per_typ_name:
        return p[p[per_typ_name].astype(str).str.contains("Driver", case=False, na=False)]

    print("[WARN] Could not find a driver indicator (PER_TYP / SEAT_POS). Keeping all persons for now.")
    return p

In [3]:
def merge_year(year: int) -> pd.DataFrame:
    p = pd.read_csv(BASE / str(year) / "PERSON.csv",
                encoding="latin1",
                low_memory=False)
    v = pd.read_csv(BASE / str(year) / "VEHICLE.csv",
                encoding="latin1",
                low_memory=False)
    a = pd.read_csv(BASE / str(year) / "ACCIDENT.csv",
                encoding="latin1",
                low_memory=False)

    # Filter to drivers
    p = filter_drivers(p)

    # Detect join keys
    case_col_p = detect_col(p, ["CASENUM"])
    case_col_v = detect_col(v, ["CASENUM"])
    case_col_a = detect_col(a, ["CASENUM"])

    veh_col_p  = detect_col(p, ["VEH_NO", "VEHNO", "VEH_NUM", "VEHNUM"])
    veh_col_v  = detect_col(v, ["VEH_NO", "VEHNO", "VEH_NUM", "VEHNUM"])

    if not (case_col_p and case_col_v and case_col_a):
        raise ValueError(f"{year}: CASENUM not found in one of the tables.")

    # PERSON -> VEHICLE merge
    if veh_col_p and veh_col_v:
        df = p.merge(
            v,
            left_on=[case_col_p, veh_col_p],
            right_on=[case_col_v, veh_col_v],
            how="left",
            suffixes=("", "_veh"),
        )
    else:
        print(f"[WARN] {year}: vehicle number key not found; merging PERSON->VEHICLE on CASENUM only.")
        df = p.merge(v, left_on=case_col_p, right_on=case_col_v, how="left", suffixes=("", "_veh"))

    # Add ACCIDENT columns (CRASH-level)
    df = df.merge(a, left_on=case_col_p, right_on=case_col_a, how="left", suffixes=("", "_acc"))

    df["YEAR"] = year
    return df

# Run yearly merges + save
for y in YEARS:
    dfy = merge_year(y)
    print(f"{y}: merged shape = {dfy.shape}")
    dfy.to_csv(OUT / f"merged_{y}.csv", index=False)

2019: merged shape = (96488, 352)
2020: merged shape = (94500, 352)
2021: merged shape = (95551, 352)
2022: merged shape = (94510, 356)
2023: merged shape = (87262, 358)


In [4]:
import pandas as pd
from pathlib import Path

INP = Path("../data/intermediate")

df2019 = pd.read_csv(INP / "merged_2019.csv", low_memory=False)
df2019[["YEAR", "CASENUM", "INJSEV_IM", "INJ_SEV"]].head()

,YEAR,CASENUM,INJSEV_IM,INJ_SEV
0,2019,201901174219,0,0
1,2019,201901176655,0,0
2,2019,201901176655,0,0
3,2019,201901176667,2,2
4,2019,201901176667,1,1


In [5]:
#Some columns are new/specific/renamed for one year only, reformat data so only variables present in all years (2019–2023) were retained
import pandas as pd
from pathlib import Path

INP = Path("../data/intermediate")
years = [2019, 2020, 2021, 2022, 2023]

dfs = [pd.read_csv(INP / f"merged_{y}.csv", low_memory=False) for y in years]

common_cols = sorted(set.intersection(*(set(df.columns) for df in dfs)))

df_all = pd.concat(
    [df[common_cols] for df in dfs],
    ignore_index=True
)

df_all.shape

(468311, 316)

In [6]:
# Check if the columns needed for the model are present and not dropped
important_cols = [
    "YEAR", "CASENUM",
    "INJ_SEV", "INJSEV_IM",
    "FATAL", "INJURY",
    "PER_TYP", "VEH_NO",
    "MONTH", "DAY_WEEK", "HOUR",
    "STATE",
    "DRINKING", "ALCOHOL", "SPEEDING",
    "AGE", "SEX"
]

[c for c in important_cols if c not in common_cols]

['FATAL', 'INJURY', 'STATE', 'SPEEDING']

In [7]:
#Fatal and Injury should be coded under severity level, check state and speeding
[c for c in common_cols if "STATE" in c]

[]

In [8]:
[c for c in common_cols if "SPEED" in c]

['SPEEDREL', 'SPEEDRELNAME']

In [9]:
#State data not present, so will have to treat this as nation-level data
#Speed is coded under other columns